In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
import pandas as pd
import json
import numpy as np
import io
from Web.browse_edgar import *
import datetime

https://www.sec.gov/ixviewer/ix.html?doc= (/Archives/edgar/data/320193/000162828017000717/a10-qq1201712312016.htm) is prefix for xlbr viewer :)


# Trying sec website. not actual filings

https://www.macrotrends.net/stocks/charts/META/meta-platforms/financial-statements
go to all financial statements
interactive data :)

10-q - to co szukamy kwartalne
10-k - to co szukamy roczne

In [2]:
def scrape_income_statement(driver):
    # TODO: https://stackoverflow.com/questions/8474031/case-insensitive-xpath-contains-possible
    table = open_income_statement(driver)
    lines = []
    for tr in table.find_elements(By.XPATH, "./tr"):
        line = []
        for th in tr.find_elements(By.XPATH, "./th") or []:
            text = th.text.strip()
            if text:
                line.append(th.text)
        for td in tr.find_elements(By.XPATH, "./td") or []:
            text = td.text.strip()
            if text:
                line.append(td.text)
        lines.append(line)
        
    title = lines.pop(0)
    try:
        date = pd.to_datetime(lines[0][0], format='%b. %d, %Y')
        dates = lines.pop(0)
    except:
        dates = title[1:]
        print('No date in second line')
    
    try:
        lines = list(filter(lambda x: len(x) > 1, lines))
        
        lines = np.array(lines)
        lines = np.char.replace(lines, '$', '')
        np.char.strip(lines)
        df = pd.DataFrame(lines, columns=['name', *dates])
    except:
        print(lines)
        return None
    return df.set_index('name')
    # return df

In [3]:
parsed_income_statements = []
def income_statement_to_df(driver):
    table = open_income_statement(driver)
    f = io.StringIO('<table>' + table.get_attribute('innerHTML') + '</table>')
    dfs = pd.read_html(f)
    parsed_income_statements.append(dfs)
    return clean_from_html_weird_tables(dfs[0])

    # if len(dfs) > 1:
        # dodaj sprawdzenie który parsować
    # return dfs[0]

In [4]:
# parsed_income_statements[3][0]
# clean_from_html_weird_tables(parsed_income_statements[3][0])

In [5]:
import functools
# parsed_income_statements[10][0]
# df: pd.DataFrame = parsed_income_statements[12][0].copy()

def clean_from_html_weird_tables(df):
# pusty nagłówek - ten dziwny nagłówek z dwóch wierszy
    nan_df = df.isna().any(axis=1)
    while nan_df[0]:
        df = df.drop(0).reset_index(drop=True)
        nan_df = df.isna().any(axis=1)

    # 12 Months Ended jako wiersz
    to_del = False
    for v in df.iloc[0]:
        if isinstance(v, str) and v.strip() == '12 Months Ended':
            to_del = True
    if to_del:  
        print('aaaa')
        df = df.drop(0)
        
    #     change to contains or something!
    columns_to_drop = df[df == '[1]'].any(axis=0)
    for i, v in enumerate(columns_to_drop):
        if v:
            df.drop(i, axis=1, inplace=True)
    
    return df


In [6]:
def cleanup_df(df):
    # remove multiIndex
    df.to_csv('df.csv')
    with open('df.csv', "r+") as f:
        d = f.readlines()
        f.seek(0)
        for i, line in enumerate(d):
            if i != 0:
                f.write(line)
        f.truncate()
    df = pd.read_csv('df.csv')
    df = df.drop(df.columns[0], axis=1)
    # not sure 
    df = df.dropna()
    
    df = df.rename(columns={df.columns[0]: 'date'})
    df = df.set_index(df.columns[0])
    df = df.map(lambda x: x.replace('$', '').replace(',', '') if isinstance(x, str) and '$' in x else x)
    df = df.transpose()
    
    return df

In [7]:
def use_search_page(driver):
    buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Interactive Data')]")
    dfs = []
    print(len(buttons))
    for i in range(len(buttons)):
        # try:
            new_buttons = driver.find_elements(By.XPATH, "//a[contains(text(), 'Interactive Data')]")
            new_buttons[i].click()
            sleep(1)
            open_sections(driver)
            df = income_statement_to_df(driver)
            df = cleanup_df(df)   
            dfs.append(df)
            driver.execute_script("window.history.go(-1)")
            driver.execute_script("window.history.go(-1)")
            print("done one")
            sleep(1)
        # except:
        #     continue
    return dfs

In [8]:
def merge_dfs(dfs):
    acc = {}
    for i, df in enumerate(dfs):
        for row in df.iterrows():
            acc[row[0]] = row[1].to_dict()
    return pd.DataFrame(acc)

In [9]:
lastDf = None

In [10]:
driver = webdriver.Chrome()
search_url = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=$$$&type=10-K&dateb=&owner=exclude&count=100'
# tickers = ['META', 'AMZN', 'AAPL']
tickers = ["AAPL", 'MSFT', 'NVDA', 'GOOG', 'AMZN', 'META', 'BRK-B', 'LLY', 'TSLA', 'AVGO'] 
# tickers = ['MSFT']
companies = {}
for ticker in tickers:
    url = search_url.replace('$$$', ticker)
    print(url)
    driver.get(url)
    dfs = use_search_page(driver)
    resultDf = merge_dfs(dfs)
    resultDf.to_csv(ticker + ".csv")

print("SAVED TO CSV")


https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=AAPL&type=10-K&dateb=&owner=exclude&count=100
16
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
done one
https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=MSFT&type=10-K&dateb=&owner=exclude&count=100
14
done clicking
done one
done clicking
done one
done clicking
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
aaaa
done one
done clicking
done one
done clicking
done one
https://www.sec.gov/cgi-bin/browse-edgar?

Exception: No table found